In [1]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import os
import beneficialUseDictionary
from utilityFunctions import *

In [2]:
# working directory
working_dir = "./ProcessedInputData"
os.chdir(working_dir)

In [3]:
# Input files
fileInput1 = "wr_v_pod_public_xy.csv" 

# output water sources
out_put = "watersources.csv"

In [4]:
##### WaDE columns

columns=['WaterSourceUUID', 'WaterSourceNativeID',	'WaterSourceName', 'WaterSourceTypeCV',
         'WaterQualityIndicatorCV',	'GNISFeatureNameCV', 'Geometry']

dtypesx = ['BigInt	NVarChar(250)	NVarChar(250)	NVarChar(250)	NVarChar(100)	NVarChar(100)',
           'NVarChar(250)	Geometry']

In [5]:
### target dataFrame

# TODO: assumes dtypes inferred from CO file
outdf100=pd.DataFrame(columns=columns)

In [6]:
print("Reading inputs...")


df100 = pd.read_csv(fileInput1,encoding = "ISO-8859-1") #, or alternatively encoding = "utf-8"

print (len(df100.index))

#df100 = df100.head(10000) #only runs first 100 lines for testing.

#df100 = df100.replace('', np.nan)
df100.head(5)

Reading inputs...


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (4,14,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


184377


,X,Y,OBJECTID,pod_display,pod_display_short,wris_link,snp_id,pod_location_id,pod_use_id,app_char,...,begin_month,begin_day,end_month,end_day,technician_initials,agency,rec_creation_date,last_updt_date,feature_quality_code,remarks
0,5.394125e+05,416705.799869,1,Permit: G 10961 * MI,G 10961,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,21755,6909,26859,G,...,1.0,1.0,12.0,31.0,MIGRT,OWRD,1996-06-01T00:00:00.000,1996-06-01T00:00:00.000,NaN,0 G 10961 1
1,5.392329e+05,416251.991798,2,Permit: G 10961 * MI,G 10961,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,21755,6910,26860,G,...,1.0,1.0,12.0,31.0,MIGRT,OWRD,1996-06-01T00:00:00.000,1996-06-01T00:00:00.000,NaN,0 G 10961 2
2,4.709328e+05,355915.458333,3,Permit: G 12684 * MI,G 12684,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,23327,9355,29682,G,...,1.0,1.0,12.0,31.0,MIGRT,OWRD,2001-06-01T00:00:00.000,2001-06-01T00:00:00.000,NaN,0 G 12684 1
3,1.010124e+06,931993.466535,4,Permit: G 12750 * MI,G 12750,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,23390,9480,29835,G,...,1.0,1.0,12.0,31.0,MIGRT,OWRD,2001-05-01T00:00:00.000,2001-05-01T00:00:00.000,NaN,0 G 12750 1
4,1.010124e+06,931993.466535,5,Permit: G 12750 * MI,G 12750,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,23390,9480,29836,G,...,1.0,1.0,12.0,31.0,MIGRT,OWRD,2001-05-01T00:00:00.000,2001-05-01T00:00:00.000,NaN,0 G 12750 1


In [7]:
list(df100.columns)

['X',
 'Y',
 'OBJECTID',
 'pod_display',
 'pod_display_short',
 'wris_link',
 'snp_id',
 'pod_location_id',
 'pod_use_id',
 'app_char',
 'app_nbr',
 'permit_char',
 'permit_nbr',
 'cert_nbr',
 'claim_char',
 'claim_nbr',
 'decree_title',
 'transfer_nbr',
 'wr_type',
 'name_last',
 'name_first',
 'name_company',
 'pod_nbr',
 'pod_char',
 'source_type',
 'use_code',
 'use_category',
 'use_code_description',
 'priority_date',
 'duty',
 'rate_cfs',
 'rate_cfs_est',
 'max_rate_cfs',
 'acre_feet',
 'acre_feet_est',
 'max_rate_acre_feet',
 'source',
 'tributary_to',
 'streamcode',
 'stream_name',
 'supplemental',
 'begin_month',
 'begin_day',
 'end_month',
 'end_day',
 'technician_initials',
 'agency',
 'rec_creation_date',
 'last_updt_date',
 'feature_quality_code',
 'remarks']

In [8]:
print("WaterSourceTypeCV...")

WRSourceTypeCVDictOR = {
    "ST":"storage",
    "SW":"surface water",
    "GW":"groundwater"
}

def assignWRSourceTypeCVOR(colrowValue):
    # may need to modify capitalization in beneficialUseDictionary
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = 'Unknown'
    else:
        keyStr = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WRSourceTypeCVDictOR[keyStr]
        except:
            outList = 'Unknown'

    return outList


df100 = df100.assign(WaterSourceTypeCV='')  #add new column and make it empty

df100['WaterSourceTypeCV'] = df100.apply(lambda row: 
                                               assignWRSourceTypeCVOR(row['wr_type']), axis=1)


df100

WaterSourceTypeCV...


,X,Y,OBJECTID,pod_display,pod_display_short,wris_link,snp_id,pod_location_id,pod_use_id,app_char,...,begin_day,end_month,end_day,technician_initials,agency,rec_creation_date,last_updt_date,feature_quality_code,remarks,WaterSourceTypeCV
0,5.394125e+05,4.167058e+05,1,Permit: G 10961 * MI,G 10961,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,21755,6909,26859,G,...,1.0,12.0,31.0,MIGRT,OWRD,1996-06-01T00:00:00.000,1996-06-01T00:00:00.000,NaN,0 G 10961 1,groundwater
1,5.392329e+05,4.162520e+05,2,Permit: G 10961 * MI,G 10961,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,21755,6910,26860,G,...,1.0,12.0,31.0,MIGRT,OWRD,1996-06-01T00:00:00.000,1996-06-01T00:00:00.000,NaN,0 G 10961 2,groundwater
2,4.709328e+05,3.559155e+05,3,Permit: G 12684 * MI,G 12684,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,23327,9355,29682,G,...,1.0,12.0,31.0,MIGRT,OWRD,2001-06-01T00:00:00.000,2001-06-01T00:00:00.000,NaN,0 G 12684 1,groundwater
3,1.010124e+06,9.319935e+05,4,Permit: G 12750 * MI,G 12750,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,23390,9480,29835,G,...,1.0,12.0,31.0,MIGRT,OWRD,2001-05-01T00:00:00.000,2001-05-01T00:00:00.000,NaN,0 G 12750 1,groundwater
4,1.010124e+06,9.319935e+05,5,Permit: G 12750 * MI,G 12750,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,23390,9480,29836,G,...,1.0,12.0,31.0,MIGRT,OWRD,2001-05-01T00:00:00.000,2001-05-01T00:00:00.000,NaN,0 G 12750 1,groundwater
5,2.098075e+06,1.005065e+06,6,Permit: G 12779 * MI,G 12779,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,23418,9515,29873,G,...,1.0,12.0,31.0,KLS,OWRD,2007-10-26T12:42:33.000,NaN,10.0,Automapped as center of the envelope for the T...,groundwater
6,2.070442e+06,9.816352e+05,7,Permit: G 13450 * MI,G 13450,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,24080,10663,31263,G,...,1.0,12.0,31.0,KLS,OWRD,2007-10-26T12:42:33.000,NaN,10.0,Automapped as center of the envelope for the T...,groundwater
7,2.071930e+06,9.777117e+05,8,Permit: G 13450 * MI,G 13450,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,24080,10664,31264,G,...,1.0,12.0,31.0,KLS,OWRD,2007-10-26T12:42:33.000,NaN,10.0,Automapped as center of the envelope for the T...,groundwater
8,2.066681e+06,9.431072e+05,9,Permit: G 13450 * MI,G 13450,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,24080,10665,31265,G,...,1.0,12.0,31.0,KLS,OWRD,2007-10-26T12:42:33.000,NaN,10.0,Automapped as center of the envelope for the T...,groundwater
9,2.067942e+06,9.685740e+05,10,Permit: G 13450 * MI,G 13450,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,24080,10666,31266,G,...,1.0,12.0,31.0,MIGRT,OWRD,2001-05-01T00:00:00.000,2001-05-01T00:00:00.000,NaN,0 G 13450 4,groundwater


In [9]:
print("WaterSourceName ...")

df100 = df100.assign(WaterSourceName='')  #add new column and make it empty

df100['WaterSourceName']=df100.apply(lambda row: 'Unspecified' if str(row['source']).strip() == ''
                                       else str(row['source']).strip(), axis=1)

df100

WaterSourceName ...


,X,Y,OBJECTID,pod_display,pod_display_short,wris_link,snp_id,pod_location_id,pod_use_id,app_char,...,end_month,end_day,technician_initials,agency,rec_creation_date,last_updt_date,feature_quality_code,remarks,WaterSourceTypeCV,WaterSourceName
0,5.394125e+05,4.167058e+05,1,Permit: G 10961 * MI,G 10961,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,21755,6909,26859,G,...,12.0,31.0,MIGRT,OWRD,1996-06-01T00:00:00.000,1996-06-01T00:00:00.000,NaN,0 G 10961 1,groundwater,FORMOSA 1 ADIT
1,5.392329e+05,4.162520e+05,2,Permit: G 10961 * MI,G 10961,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,21755,6910,26860,G,...,12.0,31.0,MIGRT,OWRD,1996-06-01T00:00:00.000,1996-06-01T00:00:00.000,NaN,0 G 10961 2,groundwater,SILVER BUTTE 1 ADIT
2,4.709328e+05,3.559155e+05,3,Permit: G 12684 * MI,G 12684,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,23327,9355,29682,G,...,12.0,31.0,MIGRT,OWRD,2001-06-01T00:00:00.000,2001-06-01T00:00:00.000,NaN,0 G 12684 1,groundwater,A WELL
3,1.010124e+06,9.319935e+05,4,Permit: G 12750 * MI,G 12750,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,23390,9480,29835,G,...,12.0,31.0,MIGRT,OWRD,2001-05-01T00:00:00.000,2001-05-01T00:00:00.000,NaN,0 G 12750 1,groundwater,A WELL
4,1.010124e+06,9.319935e+05,5,Permit: G 12750 * MI,G 12750,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,23390,9480,29836,G,...,12.0,31.0,MIGRT,OWRD,2001-05-01T00:00:00.000,2001-05-01T00:00:00.000,NaN,0 G 12750 1,groundwater,A WELL
5,2.098075e+06,1.005065e+06,6,Permit: G 12779 * MI,G 12779,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,23418,9515,29873,G,...,12.0,31.0,KLS,OWRD,2007-10-26T12:42:33.000,NaN,10.0,Automapped as center of the envelope for the T...,groundwater,A WELL
6,2.070442e+06,9.816352e+05,7,Permit: G 13450 * MI,G 13450,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,24080,10663,31263,G,...,12.0,31.0,KLS,OWRD,2007-10-26T12:42:33.000,NaN,10.0,Automapped as center of the envelope for the T...,groundwater,WELL 1
7,2.071930e+06,9.777117e+05,8,Permit: G 13450 * MI,G 13450,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,24080,10664,31264,G,...,12.0,31.0,KLS,OWRD,2007-10-26T12:42:33.000,NaN,10.0,Automapped as center of the envelope for the T...,groundwater,WELL 2
8,2.066681e+06,9.431072e+05,9,Permit: G 13450 * MI,G 13450,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,24080,10665,31265,G,...,12.0,31.0,KLS,OWRD,2007-10-26T12:42:33.000,NaN,10.0,Automapped as center of the envelope for the T...,groundwater,WELL 3
9,2.067942e+06,9.685740e+05,10,Permit: G 13450 * MI,G 13450,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,24080,10666,31266,G,...,12.0,31.0,MIGRT,OWRD,2001-05-01T00:00:00.000,2001-05-01T00:00:00.000,NaN,0 G 13450 4,groundwater,WELL 4


In [10]:
print ("Directly mapped columns")

destCols=['WaterSourceName', 'WaterSourceTypeCV']
srsCols=['WaterSourceName', 'WaterSourceTypeCV']

outdf100[destCols] = df100[srsCols]

Directly mapped columns


In [11]:
print("Hard coded ...")
outdf100.WaterQualityIndicatorCV = "Fresh"

Hard coded ...


In [12]:
print("Dropping duplicates...")

print(len(outdf100.index))
outdf100 = outdf100.drop_duplicates(subset=['WaterSourceName','WaterSourceTypeCV']) #,'WaterQualityIndicatorCV'
outdf100 = outdf100.reset_index(drop=True)

print(len(outdf100.index))

outdf100

Dropping duplicates...
184377
16878


,WaterSourceUUID,WaterSourceNativeID,WaterSourceName,WaterSourceTypeCV,WaterQualityIndicatorCV,GNISFeatureNameCV,Geometry
0,NaN,NaN,FORMOSA 1 ADIT,groundwater,Fresh,NaN,NaN
1,NaN,NaN,SILVER BUTTE 1 ADIT,groundwater,Fresh,NaN,NaN
2,NaN,NaN,A WELL,groundwater,Fresh,NaN,NaN
3,NaN,NaN,WELL 1,groundwater,Fresh,NaN,NaN
4,NaN,NaN,WELL 2,groundwater,Fresh,NaN,NaN
5,NaN,NaN,WELL 3,groundwater,Fresh,NaN,NaN
6,NaN,NaN,WELL 4,groundwater,Fresh,NaN,NaN
7,NaN,NaN,EAST FORK GRANDE RONDE RIVER,surface water,Fresh,NaN,NaN
8,NaN,NaN,LITTLE MEADOW CREEK,surface water,Fresh,NaN,NaN
9,NaN,NaN,NORTH FORK BRIDGE CREEK,surface water,Fresh,NaN,NaN


In [13]:
print ("Water source native id and UUID")

#9.12.19 Adel: For water sources table, how about we do an incremental ID? like 1, 2, 3 etc?
outdf100 = outdf100.reset_index(drop=True)
outdf100['WaterSourceNativeID'] = range(1, len(outdf100.index) + 1)

print("Adding UUID...")
outdf100['WaterSourceUUID'] = outdf100.apply(lambda row: 
                                        "_".join(["OR", str(row['WaterSourceNativeID'])]), axis=1)

outdf100 = outdf100.replace(np.nan, '')

outdf100

Water source native id and UUID
Adding UUID...


,WaterSourceUUID,WaterSourceNativeID,WaterSourceName,WaterSourceTypeCV,WaterQualityIndicatorCV,GNISFeatureNameCV,Geometry
0,OR_1,1,FORMOSA 1 ADIT,groundwater,Fresh,,
1,OR_2,2,SILVER BUTTE 1 ADIT,groundwater,Fresh,,
2,OR_3,3,A WELL,groundwater,Fresh,,
3,OR_4,4,WELL 1,groundwater,Fresh,,
4,OR_5,5,WELL 2,groundwater,Fresh,,
5,OR_6,6,WELL 3,groundwater,Fresh,,
6,OR_7,7,WELL 4,groundwater,Fresh,,
7,OR_8,8,EAST FORK GRANDE RONDE RIVER,surface water,Fresh,,
8,OR_9,9,LITTLE MEADOW CREEK,surface water,Fresh,,
9,OR_10,10,NORTH FORK BRIDGE CREEK,surface water,Fresh,,


In [14]:
print("Unknown source name")

outdf100.loc[outdf100['WaterSourceName']=='','WaterSourceName'] = "Unspecified"

outdf100

Unknown source name


,WaterSourceUUID,WaterSourceNativeID,WaterSourceName,WaterSourceTypeCV,WaterQualityIndicatorCV,GNISFeatureNameCV,Geometry
0,OR_1,1,FORMOSA 1 ADIT,groundwater,Fresh,,
1,OR_2,2,SILVER BUTTE 1 ADIT,groundwater,Fresh,,
2,OR_3,3,A WELL,groundwater,Fresh,,
3,OR_4,4,WELL 1,groundwater,Fresh,,
4,OR_5,5,WELL 2,groundwater,Fresh,,
5,OR_6,6,WELL 3,groundwater,Fresh,,
6,OR_7,7,WELL 4,groundwater,Fresh,,
7,OR_8,8,EAST FORK GRANDE RONDE RIVER,surface water,Fresh,,
8,OR_9,9,LITTLE MEADOW CREEK,surface water,Fresh,,
9,OR_10,10,NORTH FORK BRIDGE CREEK,surface water,Fresh,,


In [15]:
#write out
outdf100.to_csv(out_put, index=False, encoding = "utf-8")
print("Done watersources")

Done watersources
